# Tarea 2: Procesamiento de Lenguaje Natural



Integrantes:

* Sebastián Alday (Rut: 18294095-K)
* Paula Canales G. (Rut: 18845155-1)
* Álvaro Neira R. (Rut: 13757209-5)
* Matias Rodriguez U. (Rut: 18362815-1)

# Introducción

In [34]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

# 1. Creacion y limpieza de la base de datos 

## 1.1 Elija una organización que cuenta con una lista de preguntas y respuestas

Chilexpress
https://centrodeayuda.chilexpress.cl/home

## 1.2 Genere una tabla (tablaQA.xls) con las preguntas y respuestas de la organización seleccionada.

In [1]:
!wget https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv

--2021-12-28 06:26:09--  https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9zty4fuh8boofda/tablaQA.csv [following]
--2021-12-28 06:26:09--  https://www.dropbox.com/s/raw/9zty4fuh8boofda/tablaQA.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde1f2a001bbc0c490f1392c644.dl.dropboxusercontent.com/cd/0/inline/BcqayjUIxshIIhcnnRCxHoS6D8ZigGuszCqdYcz33wi61LyOHkwd5VKiUu19TGOj7ZkjrKVirStGNvPy6RpmRjgCmzBEVaFvFZ1YepJISDkszL7ehTxPat0nN256nHF_rDQHFKYLJVU3U6u57vQHrBBC/file# [following]
--2021-12-28 06:26:09--  https://ucde1f2a001bbc0c490f1392c644.dl.dropboxusercontent.com/cd/0/inline/BcqayjUIxshIIhcnnRCxHoS6D8ZigGuszCqdYcz33wi61LyOHkwd5VKiUu19TGOj7ZkjrKVirStGNvPy6RpmRjgCmzBEV

In [38]:
import pandas as pd
qa = pd.read_csv('/content/tablaQA.csv', sep=';')
qa.tail(45)

,Pregunta,Respuesta
1,¿Cuál es el plazo para retirar un envío desde ...,El plazo para retirar un envío desde una sucur...
2,¿Puede otra persona retirar mi envío de una su...,Para que una persona que no es el destinatario...
3,¿Se puede modificar la dirección de envío?,"Sí, el cambio puede ser solicitado por el remi..."
4,¿Puedo cambiar el destino del envío desde una ...,"Sí, el cambio puede ser solicitado por el remi..."
5,¿El destinatario puede cambiar la dirección de...,"No es posible. Sin embargo, si un envío no pue..."
6,Si el destinatario no se encuentra en la direc...,"Sí, puede recibirlo cualquier persona adulta q..."
7,¿Qué pasa si el destinatario no puede recibir ...,Chilexpress realiza dos intentos de entrega en...
8,¿Cuántos intentos de entrega en el domicilio r...,Chilexpress realiza dos intentos de entrega en...
9,¿Chilexpress solicita el RUT del destinatario ...,"No es necesario para un envío nacional, pero d..."
10,¿Qué es un envío sobredimensionado?,Es toda encomienda que no cumpla el estándar e...


In [3]:
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pregunta   46 non-null     object
 1   Respuesta  46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


## 1.3 Genere una tabla (tiposmensajes.xls) con ejemplos de las siguientes clases de mensaje: "saludo", "despedida", "nombre", "informacion"

In [39]:
!wget https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv

--2021-12-28 06:58:10--  https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv [following]
--2021-12-28 06:58:10--  https://www.dropbox.com/s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc209972176e80b0affefb358cbe.dl.dropboxusercontent.com/cd/0/inline/BcpkIJvqstCWgOpngRyJaJpjdpBkQBW_Rvpqcs9k7JAHkJcHSMRz8HjWbcqKvz8OfSctmBFzDEGxRbjn-DLT2OQt7QW501AZeYqVYH5R1GcfDlYddjn6O-JB-_ETWWGyd6nIjES8hTc75IOSymRGUzjR/file# [following]
--2021-12-28 06:58:10--  https://uc209972176e80b0affefb358cbe.dl.dropboxusercontent.com/cd/0/inline/BcpkIJvqstCWgOpngRyJaJpjdpBkQBW_Rvpqcs9k7JAHkJcHSMRz8HjWbcqKvz8OfSctmBFzDEG

In [42]:
messages = pd.read_csv('/content/tiposmensajes.csv', sep=',')
messages.tail(59)

,Clase,Mensaje
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas
5,despedida,Chao
6,despedida,Adiós
7,despedida,Hasta luego
8,despedida,Nos vemos
9,despedida,Bye
10,nombre,Mi nombre es


## 1.4 Genere respuestas predeterminadas (respuestasDefecto.xls) para los mensajes de tipo "saludo", "despedida", "nombre"

In [51]:
!wget https://github.com/alvaro-neira/nlp-homework2/blob/main/respuestasDefecto.csv -O /content/respuestasDefecto.csv

--2021-12-28 07:09:45--  https://github.com/alvaro-neira/nlp-homework2/blob/main/respuestasDefecto.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/respuestasDefecto.csv’

/content/respuestas     [ <=>                ] 162.47K  --.-KB/s    in 0.02s   

2021-12-28 07:09:45 (7.25 MB/s) - ‘/content/respuestasDefecto.csv’ saved [166372]



In [52]:
answers = pd.read_csv('/content/respuestasDefecto.csv', sep=',')
answers.tail(12)

ParserError: ignored

## 1.5 Describa en términos generales las tablas que construyó.



*   Tabla QA: Se construyó en base a las preguntas y respuestas del centro de ayuda de Chilexpress https://centrodeayuda.chilexpress.cl/home . Se extrajo cada pregunta y respuesta de cada categoría (Envíos Nacionales, Envíos y retiros en App, Envíos internacionales, Giros en Chile y el mundo, Atención en sucursal, Cuenta empresa) y se eliminaron las preguntas repetidas. 
*   Tipos de mensajes: se construyó con mensajes tipo de saludo, despedida y nombre. Para la información se utilizó las preguntas obtenidas en la Tabla QA.
*   Respuestas por defecto: se construyó con respuestas tipo de saludo, despedida y nombre de nuestro bot Eva



# 2. Análisis de distancia

Para el primer chatbot utilizará una métrica de su elección para responder
a cada pregunta/texto del usuario.

## 2.1 Utilice algún embeddings utilizado en el curso para codificar el texto de entrada

### 2.1.1 Word2Vec

In [12]:
import nltk
import gensim
import re
import pandas as pd
import sklearn
import numpy as np

import sklearn.linear_model
import sklearn.model_selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [13]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  #if len(tokens)==1:
  #  tokens=text
  return tokens

def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

In [14]:
corpus = messages.Mensaje.tolist()
corpus_preprocessed = list(map(preprocessor,corpus))


In [15]:
model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed)


### 2.1.2 ELMO

In [16]:
!pip install overrides==3.1.0
!pip install elmoformanylangs

  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10186 sha256=742f5eee485daca4c38e22688cf2ef7c41faea29912d36f4e9fb6fd71bb61b05
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
Successfully built overrides
     |████████████████████████████████| 42 kB 955 kB/s 


In [17]:
!wget http://vectors.nlpl.eu/repository/11/145.zip
!unzip 145.zip

--2021-12-28 06:31:52--  http://vectors.nlpl.eu/repository/11/145.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 419687058 (400M) [application/zip]
Saving to: ‘145.zip’

145.zip             100%[===================>] 400.24M  18.6MB/s    in 23s     

2021-12-28 06:32:16 (17.2 MB/s) - ‘145.zip’ saved [419687058/419687058]

Archive:  145.zip
  inflating: char.dic                
  inflating: config.json             
  inflating: encoder.pkl             
  inflating: meta.json               
  inflating: README                  
  inflating: token_embedder.pkl      
  inflating: word.dic                


In [18]:
import elmoformanylangs

In [19]:
e = elmoformanylangs.Embedder('')

2021-12-28 06:32:31,135 WARNING: Could not find config.  Trying cnn_50_100_512_4096_sample.json
2021-12-28 06:32:31,138 WARNING: Could not find config.  Trying /usr/local/lib/python3.7/dist-packages/elmoformanylangs/configs/cnn_50_100_512_4096_sample.json
2021-12-28 06:32:31,219 INFO: char embedding size: 2637
2021-12-28 06:32:32,312 INFO: word embedding size: 185214
2021-12-28 06:32:44,274 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(185214, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2637, 50, padding_idx=2634)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel

In [20]:
sentences = list(map(preprocessor,messages.Mensaje))
features_raw = e.sents2elmo(sentences)

2021-12-28 06:32:53,953 INFO: 1 batches, avg len: 10.8


## 2.2 Con el texto de entrada codificado y usando embeddings proponga una manera de identificar la clase del texto de entrada.

In [21]:
def predict_class(question):
  tokens = preprocessor(question)
  vector = vectorizer(tokens,model).reshape(1, -1)
  return clf.predict(vector)

### 2.2.1 Word2Vec

In [22]:
messages

,Clase,Mensaje
0,saludo,Hola
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas
5,despedida,Chao
6,despedida,Adiós
7,despedida,Hasta luego
8,despedida,Nos vemos
9,despedida,Bye


In [23]:
features = np.zeros(shape=(len(messages),model.wv.vectors.shape[1]))
for i,msg in enumerate(messages.Mensaje):
  features[i,:] = vectorizer(preprocessor(msg),model)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.7666666666666667

In [24]:
#clf = sklearn.linear_model.LogisticRegression()
#clf.fit(features,messages.Clase)
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(features,np.array(messages.Clase))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [25]:
vectorprueba = vectorizer(preprocessor("Hola buenas"),model).reshape(1, -1)
vectorprueba

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


array([[0.]])

In [26]:
preprocessor("Hola buenas")

['hola', 'buenas']

In [27]:
vectorsprueba = []
for i in ['mi',"nombre","es"]:
    try:
        vectorsprueba.append(model.wv[i])

        #print(i)
        #print(vectorsprueba)
    except:
        pass
print(model.wv["mi"])
#print(vectorsprueba)
#print(np.nan_to_num(np.mean(vectorsprueba,axis=0)))

[-1.6995457e-03 -9.2877977e-04  1.4382645e-03  6.6863821e-04
 -4.4397870e-03 -3.1323459e-03 -1.8924028e-04 -4.8062871e-03
  1.9674178e-03  4.3180734e-03  2.4309296e-03 -3.5227091e-05
  3.0498535e-03 -2.7709205e-03  1.7612906e-03 -9.4162548e-05
  5.0396142e-05  2.1838555e-03 -4.2359806e-03  3.6459793e-03
 -1.9686059e-03 -7.3772017e-04 -4.0274179e-03 -4.3230434e-03
 -2.4793739e-03  3.8916268e-03  3.7436390e-03 -6.4668345e-04
 -2.4535090e-03  2.0095126e-03 -1.1312971e-03  3.3694546e-04
  2.7787308e-03  4.7811880e-03  6.7428315e-05 -1.3106432e-03
 -3.8366899e-04  1.9044249e-03  2.8945138e-03 -2.2313038e-03
 -1.3034992e-03  4.7142333e-03  2.8349233e-03  2.2681574e-03
 -2.0405012e-03  4.9323267e-03 -3.7786434e-03  8.3870755e-04
  4.6378993e-03  3.0539746e-03 -2.4690051e-04  4.0654968e-03
  8.9473266e-04 -4.1607022e-03 -2.3382432e-03 -1.9727917e-04
 -9.2634431e-04 -4.2552315e-03 -1.9927928e-03  1.1907857e-03
 -6.9371244e-04  4.7849882e-03 -1.1189398e-03  6.7974947e-04
  2.2912347e-03  4.43144

In [29]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Mi nombre es'))
# print(predict_class('hola buenas'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))

['información']
['nombre']
['información']
['información']


### 2.2.2 ELMO

In [30]:
features_elmo = []
for doc in features_raw:
  features_elmo.append(doc.mean(0))
features_elmo = np.vstack(features_elmo)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=1000),features_elmo,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.8833333333333334

In [31]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(features_elmo,messages.Clase)

LogisticRegression()

In [33]:
# print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
# print(predict_class('Hola buen día como estas'))
# print(predict_class('Mi nombre es Paula'))
# print(predict_class('¿puedo retirar un giro de dinero?'))
# print(predict_class('Puede otra persona retirar mi pedido'))

## 2.3 Si el texto de entrada es del tipo "información”, busque ahora la pregunta mas similar y retorne la respuesta asociada.

## 2.4 Reporte el resultado con textos de prueba.

# 3. Análisis transformer 

Para el segundo chatbot utilice un transformer (por ejemplo BERT).

## 3.1 Utilice el modelo transformer para clasificar el texto de entrada, y para extraer la respuesta de la tabla de preguntas y respuestas cuando el mensaje sea del tipo “información”.

## 3.2 Reporte el tipo de red, y las métricas de entrenamiento usadas

## 3.3 Reporte el resultado con los textos de prueba.

# Conclusión